In [30]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# Define the dataset path
data_dir = 'Data2'

# Parameters
img_height, img_width = 128, 128  # Resize images to 128x128
batch_size = 32

# Data Augmentation and Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,   # Normalize pixel values
    validation_split=0.2,  # 20% data for validation
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [31]:

# Load Training and Validation Data
train_data = data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 2029 images belonging to 9 classes.
Found 504 images belonging to 9 classes.


In [32]:
classes = 9

# Build the Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(classes, activation='softmax')  # 9 output classes
])


In [33]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [34]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

Epoch 1/10
64/64 [==============================] - 33s 501ms/step - loss: 1.8897 - accuracy: 0.3139 - val_loss: 0.9995 - val_accuracy: 0.7480
Epoch 2/10
64/64 [==============================] - 30s 467ms/step - loss: 0.8880 - accuracy: 0.7048 - val_loss: 0.3504 - val_accuracy: 0.8829
Epoch 3/10
64/64 [==============================] - 36s 555ms/step - loss: 0.5867 - accuracy: 0.7969 - val_loss: 0.2382 - val_accuracy: 0.9365
Epoch 4/10
64/64 [==============================] - 35s 544ms/step - loss: 0.4855 - accuracy: 0.8305 - val_loss: 0.2211 - val_accuracy: 0.9306
Epoch 5/10
64/64 [==============================] - 30s 470ms/step - loss: 0.3327 - accuracy: 0.8827 - val_loss: 0.1599 - val_accuracy: 0.9464
Epoch 6/10
64/64 [==============================] - 30s 473ms/step - loss: 0.3195 - accuracy: 0.8886 - val_loss: 0.1176 - val_accuracy: 0.9583
Epoch 7/10
64/64 [==============================] - 31s 478ms/step - loss: 0.2746 - accuracy: 0.9083 - val_loss: 0.1219 - val_accuracy: 0.9663

In [35]:

# Save the Model
model.save('sign_language_model_2.h5')

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {test_accuracy * 100:.2f}%")


16/16 [==============================] - 3s 201ms/step - loss: 0.1002 - accuracy: 0.9683
Validation Accuracy: 96.83%


In [36]:
from tensorflow.keras.models import load_model
model = load_model('sign_language_model_2.h5')

In [37]:
# Testing
from tensorflow.keras.preprocessing import image

def predict_class(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions)
    class_label = list(train_data.class_indices.keys())[class_idx]
    return class_label

In [38]:

# Test Example
example_image = 'Data2\Down\Image_1738594012.641176.jpg'
result = predict_class(example_image)
print(f"Predicted Class: {result}")

1/1 [==============================] - 0s 118ms/step
Predicted Class: Down


In [39]:

# Test Example
example_image = 'Data2\Wait\Image_1738593619.740003.jpg'
result = predict_class(example_image)
print(f"Predicted Class: {result}")

1/1 [==============================] - 0s 41ms/step
Predicted Class: Wait


In [41]:
# Test Example
example_image = 'Data2\Sorry\Image_1738593681.600836.jpg'
result = predict_class(example_image)
print(f"Predicted Class: {result}")

1/1 [==============================] - 0s 34ms/step
Predicted Class: Sorry
